In [4]:
import time

start_time = time.time()

In [5]:


x = 5
print('55')
y = []
for i in range(999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999):
    y.append(i)


55


In [ ]:
end_time = time.time()

print(f'Total runtime of the notebook is {end_time - start_time} seconds')

Total runtime of the notebook is 0.19400548934936523 seconds


In [ ]:
from scipy.interpolate import interp1d
from numpy import linspace

fig, ax = plt.subplots(figsize=(8,4))

fprs = []
tprs = []
aucs = []
cv_means = []

# Standard FPR values (might want to change range and/or steps)
standard_fpr = np.linspace(0, 1, 100)

for i, model in enumerate(models):

    mean_tpr = np.zeros_like(standard_fpr)

    for j in range(n_classes):
        cv_probs = cross_val_predict(model, X, y_bin[:,j], cv=folds, method='predict_proba')
        cv_prob_true = cv_probs[:,1]
        fpr, tpr, thresh = roc_curve(y_bin[:,j], cv_prob_true)
        
        roc_auc_score_var = roc_auc_score(y_bin[:,j], cv_prob_true)
        aucs.append(roc_auc_score_var)

        interp_tpr = interp1d(fpr, tpr, kind='linear', fill_value=(0,1), bounds_error=False)
        mean_tpr += interp_tpr(standard_fpr)

        cv_scores = cross_val_score(model, X, y_bin[:,j], cv=folds, scoring='accuracy')
        cv_means.append(cv_scores.mean())

    mean_tpr /= n_classes

    ax.plot(standard_fpr, mean_tpr, label=f'Model: {model.__class__.__name__} | roc_auc: {np.mean(aucs):.4f} | mean_cv: {np.mean(cv_means):.4f}')

ax.plot([0, 1], [0, 1], linestyle='--', color='gray', label='base 0.5')
ax.set_title(f'Average ROC Curves')
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()
